# Word2Vec Tutorial

from https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/word2vec.ipynb

In case you missed the buzz, word2vec is a widely featured as a member of the “new wave” of machine learning algorithms based on neural networks, commonly referred to as "deep learning" (though word2vec itself is rather shallow). Using large amounts of unannotated plain text, word2vec learns relationships between words automatically. The output are vectors, one vector per word, with remarkable linear relationships that allow us to do things like vec(“king”) – vec(“man”) + vec(“woman”) =~ vec(“queen”), or vec(“Montreal Canadiens”) – vec(“Montreal”) + vec(“Toronto”) resembles the vector for “Toronto Maple Leafs”.

Word2vec is very useful in [automatic text tagging](https://github.com/RaRe-Technologies/movie-plots-by-genre), recommender systems and machine translation.

Check out an [online word2vec demo](http://radimrehurek.com/2014/02/word2vec-tutorial/#app) where you can try this vector algebra for yourself. That demo runs `word2vec` on the Google News dataset, of **about 100 billion words**.

## This tutorial

In this tutorial you will learn how to train and evaluate word2vec models on your business data.


## Preparing the Input
Starting from the beginning, gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence is a list of words (utf8 strings):

In [1]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-06-29 17:53:00,322 : INFO : collecting all words and their counts
2017-06-29 17:53:00,323 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-29 17:53:00,325 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2017-06-29 17:53:00,326 : INFO : Loading a fresh vocabulary
2017-06-29 17:53:00,326 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-06-29 17:53:00,327 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-06-29 17:53:00,328 : INFO : deleting the raw counts dictionary of 3 items
2017-06-29 17:53:00,329 : INFO : sample=0.001 downsamples 3 most-common words
2017-06-29 17:53:00,330 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-06-29 17:53:00,331 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-06-29 17:53:00,332 : INFO : resetting layer weights
2017-06-29 17:53:00,333 : INFO : training model with 3 workers o

Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.

Gensim only requires that the input must provide sentences sequentially, when iterated over. No need to keep everything in RAM: we can provide one sentence, process it, forget it, load another sentence…

For example, if our input is strewn across several files on disk, with one sentence per line, then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

In [3]:
# create some toy data to use with the following example
import smart_open, os

if not os.path.exists('./data/'):
    os.makedirs('./data/')

filenames = ['./data/f1.txt', './data/f2.txt']

for i, fname in enumerate(filenames):
    with smart_open.smart_open(fname, 'w') as fout:
        for line in sentences[i]:
            fout.write(line + '\n')

In [4]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

In [5]:
sentences = MySentences('./data/') # a memory-friendly iterator
print(list(sentences))

[['first'], ['sentence'], ['second'], ['sentence']]


In [6]:
# generate the Word2Vec model
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-06-29 17:53:03,216 : INFO : collecting all words and their counts
2017-06-29 17:53:03,218 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-29 17:53:03,219 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-06-29 17:53:03,220 : INFO : Loading a fresh vocabulary
2017-06-29 17:53:03,221 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-06-29 17:53:03,222 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-06-29 17:53:03,223 : INFO : deleting the raw counts dictionary of 3 items
2017-06-29 17:53:03,224 : INFO : sample=0.001 downsamples 3 most-common words
2017-06-29 17:53:03,225 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-06-29 17:53:03,226 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-06-29 17:53:03,227 : INFO : resetting layer weights
2017-06-29 17:53:03,228 : INFO : training model with 3 workers o

In [7]:
print(model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'first': <gensim.models.keyedvectors.Vocab object at 0x10c4447b8>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x10af990b8>, 'second': <gensim.models.keyedvectors.Vocab object at 0x10e5b32b0>}


Say we want to further preprocess the words from the files — convert to unicode, lowercase, remove numbers, extract named entities… All of this can be done inside the `MySentences` iterator and `word2vec` doesn’t need to know. All that is required is that the input yields one sentence (list of utf8 words) after another.

**Note to advanced users:** calling `Word2Vec(sentences, iter=1)` will run **two** passes over the sentences iterator. In general it runs `iter+1` passes. By the way, the default value is `iter=5` to comply with Google's word2vec in C language. 
  1. The first pass collects words and their frequencies to build an internal dictionary tree structure. 
  2. The second pass trains the neural model.

These two passes can also be initiated manually, in case your input stream is non-repeatable (you can only afford one pass), and you’re able to initialize the vocabulary some other way:

In [8]:
# build the same model, making the 2 steps explicit
new_model = gensim.models.Word2Vec(min_count=1)  # an empty model, no training
new_model.build_vocab(sentences)                 # can be a non-repeatable, 1-pass generator     
new_model.train(sentences, total_examples=new_model.corpus_count, epochs=new_model.iter)                       
# can be a non-repeatable, 1-pass generator

2017-06-29 17:53:04,962 : INFO : collecting all words and their counts
2017-06-29 17:53:04,964 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-29 17:53:04,965 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-06-29 17:53:04,966 : INFO : Loading a fresh vocabulary
2017-06-29 17:53:04,967 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-06-29 17:53:04,968 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-06-29 17:53:04,969 : INFO : deleting the raw counts dictionary of 3 items
2017-06-29 17:53:04,970 : INFO : sample=0.001 downsamples 3 most-common words
2017-06-29 17:53:04,971 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-06-29 17:53:04,972 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-06-29 17:53:04,973 : INFO : resetting layer weights
2017-06-29 17:53:04,974 : INFO : training model with 3 workers o

0

In [9]:
print(new_model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'first': <gensim.models.keyedvectors.Vocab object at 0x10c4447b8>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x10af990b8>, 'second': <gensim.models.keyedvectors.Vocab object at 0x10e5b32b0>}


### More data would be nice
For the following examples, we'll use the [Lee Corpus](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor) (which you already have if you've installed gensim):

In [10]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'

In [11]:
class MyText(object):
    def __iter__(self):
        for line in open(lee_train_file):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

sentences = MyText()

print(sentences)

## Training
`Word2Vec` accepts several parameters that affect both training speed and quality.

### min_count
`min_count` is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

In [12]:
# default value of min_count=5
model = gensim.models.Word2Vec(sentences, min_count=10)

2017-06-29 17:53:10,343 : INFO : collecting all words and their counts
2017-06-29 17:53:10,384 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-29 17:53:10,402 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-06-29 17:53:10,403 : INFO : Loading a fresh vocabulary
2017-06-29 17:53:10,410 : INFO : min_count=10 retains 806 unique words (7% of original 10186, drops 9380)
2017-06-29 17:53:10,411 : INFO : min_count=10 leaves 40964 word corpus (68% of original 59890, drops 18926)
2017-06-29 17:53:10,414 : INFO : deleting the raw counts dictionary of 10186 items
2017-06-29 17:53:10,415 : INFO : sample=0.001 downsamples 54 most-common words
2017-06-29 17:53:10,416 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2017-06-29 17:53:10,417 : INFO : estimated required memory for 806 words and 100 dimensions: 1047800 bytes
2017-06-29 17:53:10,420 : INFO : resetting layer weights
2017-06-29 17:5

### size
`size` is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

In [13]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=200)

2017-06-29 17:53:11,598 : INFO : collecting all words and their counts
2017-06-29 17:53:11,600 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-29 17:53:11,622 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-06-29 17:53:11,623 : INFO : Loading a fresh vocabulary
2017-06-29 17:53:11,633 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-06-29 17:53:11,635 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-06-29 17:53:11,640 : INFO : deleting the raw counts dictionary of 10186 items
2017-06-29 17:53:11,642 : INFO : sample=0.001 downsamples 49 most-common words
2017-06-29 17:53:11,643 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-06-29 17:53:11,644 : INFO : estimated required memory for 1723 words and 200 dimensions: 3618300 bytes
2017-06-29 17:53:11,650 : INFO : resetting layer weights
2017-06-29 17:

### workers
`workers`, the last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [14]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

2017-06-29 17:53:12,583 : INFO : collecting all words and their counts
2017-06-29 17:53:12,585 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-29 17:53:12,607 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-06-29 17:53:12,608 : INFO : Loading a fresh vocabulary
2017-06-29 17:53:12,617 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-06-29 17:53:12,618 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-06-29 17:53:12,624 : INFO : deleting the raw counts dictionary of 10186 items
2017-06-29 17:53:12,625 : INFO : sample=0.001 downsamples 49 most-common words
2017-06-29 17:53:12,625 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-06-29 17:53:12,626 : INFO : estimated required memory for 1723 words and 100 dimensions: 2239900 bytes
2017-06-29 17:53:12,631 : INFO : resetting layer weights
2017-06-29 17:

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.

For example a syntactic analogy of comparative type is bad:worse;good:?. There are total of 9 types of syntactic comparisons in the dataset like plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as capital cities (Paris:France;Tokyo:?) or family members (brother:sister;dad:?). 

Gensim supports the same evaluation set, in exactly the same format:

In [15]:
model.accuracy('./datasets/questions-words.txt')

FileNotFoundError: [Errno 2] No such file or directory: './datasets/questions-words.txt'

This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.



In the December 2016 release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset specific to your business based on it. It contains word pairs together with human-assigned similarity judgments. It measures the relatedness or co-occurrence of two words. For example, 'coast' and 'shore' are very similar as they appear in the same context. At the same time 'clothes' and 'closet' are less similar because they are related but not interchangeable.

In [16]:
model.evaluate_word_pairs(test_data_dir + 'wordsim353.tsv')

2017-06-29 17:53:17,288 : INFO : Pearson correlation coefficient against /Users/gmelli/anaconda/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv: 0.1254
2017-06-29 17:53:17,289 : INFO : Spearman rank-order correlation coefficient against /Users/gmelli/anaconda/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv: 0.1449
2017-06-29 17:53:17,289 : INFO : Pairs with unknown words ratio: 85.6%


((0.1254291719951991, 0.38048251282405665),
 SpearmanrResult(correlation=0.14494852392259017, pvalue=0.31017770117676685),
 85.55240793201133)

Once again, **good performance on Google's or WS-353 test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task. For an example of how to use word2vec in a classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre).

## Storing and loading models
You can store/load models using the standard gensim methods:

In [17]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

2017-06-29 17:53:18,469 : INFO : saving Word2Vec object under /var/folders/th/j6jk1g_17rq53htctld_qdh5_24t68/T/tmpgkk9a8tggensim_temp, separately None
2017-06-29 17:53:18,470 : INFO : not storing attribute syn0norm
2017-06-29 17:53:18,471 : INFO : not storing attribute cum_table
2017-06-29 17:53:18,497 : INFO : saved /var/folders/th/j6jk1g_17rq53htctld_qdh5_24t68/T/tmpgkk9a8tggensim_temp


In [18]:
new_model = gensim.models.Word2Vec.load(temp_path)  # open the model

2017-06-29 17:53:18,767 : INFO : loading Word2Vec object from /var/folders/th/j6jk1g_17rq53htctld_qdh5_24t68/T/tmpgkk9a8tggensim_temp
2017-06-29 17:53:18,782 : INFO : loading wv recursively from /var/folders/th/j6jk1g_17rq53htctld_qdh5_24t68/T/tmpgkk9a8tggensim_temp.wv.* with mmap=None
2017-06-29 17:53:18,783 : INFO : setting ignored attribute syn0norm to None
2017-06-29 17:53:18,784 : INFO : setting ignored attribute cum_table to None
2017-06-29 17:53:18,785 : INFO : loaded /var/folders/th/j6jk1g_17rq53htctld_qdh5_24t68/T/tmpgkk9a8tggensim_temp


which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:
```
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip:
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)
```

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences and [new vocabulary words](online_w2v_tutorial.ipynb):

In [19]:
model = gensim.models.Word2Vec.load(temp_path)
more_sentences = [['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 'training', 'it', 'with', 'more', 'sentences']]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.iter)

# cleaning up temp
os.close(fs)
os.remove(temp_path)

2017-06-29 17:53:19,405 : INFO : loading Word2Vec object from /var/folders/th/j6jk1g_17rq53htctld_qdh5_24t68/T/tmpgkk9a8tggensim_temp
2017-06-29 17:53:19,421 : INFO : loading wv recursively from /var/folders/th/j6jk1g_17rq53htctld_qdh5_24t68/T/tmpgkk9a8tggensim_temp.wv.* with mmap=None
2017-06-29 17:53:19,422 : INFO : setting ignored attribute syn0norm to None
2017-06-29 17:53:19,423 : INFO : setting ignored attribute cum_table to None
2017-06-29 17:53:19,424 : INFO : loaded /var/folders/th/j6jk1g_17rq53htctld_qdh5_24t68/T/tmpgkk9a8tggensim_temp
2017-06-29 17:53:19,429 : INFO : collecting all words and their counts
2017-06-29 17:53:19,430 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-29 17:53:19,431 : INFO : collected 13 word types from a corpus of 13 raw words and 1 sentences
2017-06-29 17:53:19,432 : INFO : Updating model with new vocabulary
2017-06-29 17:53:19,433 : INFO : New added 0 unique words (0% of original 13)
                        and i

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `KeyedVectors.load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`Word2Vec` supports several word similarity tasks out of the box:

In [20]:
model.most_similar(positive=['human', 'crime'], negative=['party'], topn=1)

2017-06-29 17:53:19,750 : INFO : precomputing L2-norms of word weight vectors


[('perth.', 0.9931573867797852)]

In [21]:
model.doesnt_match("input is lunch he sentence cat".split())

2017-06-29 17:53:19,940 : WARNING : vectors for words {'cat', 'input', 'lunch'} are not present in the model, ignoring these words


'sentence'

In [22]:
print(model.similarity('human', 'party'))
print(model.similarity('tree', 'murder'))

0.998933711216
0.994389959033


You can get the probability distribution for the center word given the context words as input:

In [23]:
print(model.predict_output_word(['emergency', 'beacon', 'received']))

[('continue', 0.0009550039), ('more', 0.00094606681), ('can', 0.00090504414), ('training', 0.00085821631), ('it', 0.00078384532), ('australia', 0.00075342914), ('government', 0.00074237544), ('first', 0.00073976576), ('they', 0.00073577382), ('i', 0.00072868168)]


The results here don't look good because the training corpus is very small. To get meaningful results one needs to train on 500k+ words.

If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [24]:
model['tree']  # raw NumPy vector of a word

array([-0.02605367, -0.04322816, -0.00762196,  0.04137046,  0.02919012,
        0.00124471,  0.01876163,  0.04249236, -0.02101184, -0.01950756,
       -0.0137376 ,  0.00039006, -0.03117888,  0.02350005, -0.03014572,
       -0.01977978, -0.04086982,  0.00936697, -0.02998453,  0.00662421,
       -0.00939949,  0.03744794, -0.03534386, -0.00508052, -0.00881657,
        0.03726415, -0.04934629,  0.00657367,  0.06366704, -0.01057758,
       -0.02135223,  0.04067106, -0.05911547, -0.02466129,  0.02049921,
       -0.01371224,  0.02147543, -0.04598195, -0.00415067,  0.04338992,
       -0.02375613,  0.00935619,  0.05750382,  0.01922596,  0.06205679,
       -0.0032138 ,  0.00349477,  0.02970535, -0.01133251, -0.0165915 ,
       -0.01978616, -0.02736385, -0.04638084, -0.05328544, -0.04595629,
        0.02322076,  0.00742439, -0.00706375, -0.06272949,  0.01241395,
        0.005152  ,  0.00813371, -0.0102999 ,  0.0193727 ,  0.01993319,
        0.03972232, -0.01416612, -0.03258677,  0.02210148,  0.03

…or en-masse as a 2D NumPy matrix from `model.wv.syn0`.

## Training Loss Computation

The parameter `compute_loss` can be used to toggle computation of loss while training the Word2Vec model. The computed loss is stored in the model attribute `running_training_loss` and can be retrieved using the function `get_latest_training_loss` as follows : 

In [25]:
# instantiating and training the Word2Vec model
model_with_loss = gensim.models.Word2Vec(sentences, min_count=1, compute_loss=True, hs=0, sg=1, seed=42)

# getting the training loss value
training_loss = model_with_loss.get_latest_training_loss()
print(training_loss)

TypeError: __init__() got an unexpected keyword argument 'compute_loss'

### Benchmarks to see effect of training loss compuation code on training time

We first download and setup the test data used for getting the benchmarks.

In [26]:
input_data_files = []

def setup_input_data():
    # check if test data already present
    if os.path.isfile('./text8') is False:

        # download and decompress 'text8' corpus
        import zipfile
        ! wget 'http://mattmahoney.net/dc/text8.zip'
        ! unzip 'text8.zip'
    
        # create 1 MB, 10 MB and 50 MB files
        ! head -c1000000 text8 > text8_1000000
        ! head -c10000000 text8 > text8_10000000
        ! head -c50000000 text8 > text8_50000000
                
    # add 25 KB test file
    input_data_files.append(os.path.join(os.getcwd(), '../../gensim/test/test_data/lee_background.cor'))

    # add 1 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_1000000'))

    # add 10 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_10000000'))

    # add 50 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_50000000'))

    # add 100 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8'))

setup_input_data()
print(input_data_files)

/bin/sh: wget: command not found
unzip:  cannot find or open text8.zip, text8.zip.zip or text8.zip.ZIP.
head: text8: No such file or directory
head: text8: No such file or directory
head: text8: No such file or directory
['/Users/gmelli/Documents/GitHub_public/notebooks_Jupyter/Text_Data/../../gensim/test/test_data/lee_background.cor', '/Users/gmelli/Documents/GitHub_public/notebooks_Jupyter/Text_Data/text8_1000000', '/Users/gmelli/Documents/GitHub_public/notebooks_Jupyter/Text_Data/text8_10000000', '/Users/gmelli/Documents/GitHub_public/notebooks_Jupyter/Text_Data/text8_50000000', '/Users/gmelli/Documents/GitHub_public/notebooks_Jupyter/Text_Data/text8']


We now compare the training time taken for different combinations of input data and model training parameters like `hs` and `sg`.

In [27]:
# using 25 KB and 50 MB files only for generating o/p -> comment next line for using all 5 test files
input_data_files = [input_data_files[0], input_data_files[-2]]
print(input_data_files)

import time
import numpy as np
import pandas as pd

train_time_values = []
seed_val = 42
sg_values = [0, 1]
hs_values = [0, 1]

for data_file in input_data_files:
    data = gensim.models.word2vec.LineSentence(data_file) 
    for sg_val in sg_values:
        for hs_val in hs_values:
            for loss_flag in [True, False]:
                time_taken_list = []
                for i in range(3):
                    start_time = time.time()
                    w2v_model = gensim.models.Word2Vec(data, compute_loss=loss_flag, sg=sg_val, hs=hs_val, seed=seed_val) 
                    time_taken_list.append(time.time() - start_time)

                time_taken_list = np.array(time_taken_list)
                time_mean = np.mean(time_taken_list)
                time_std = np.std(time_taken_list)
                train_time_values.append({'train_data': data_file, 'compute_loss': loss_flag, 'sg': sg_val, 'hs': hs_val, 'mean': time_mean, 'std': time_std})

train_times_table = pd.DataFrame(train_time_values)
train_times_table = train_times_table.sort_values(by=['train_data', 'sg', 'hs', 'compute_loss'], ascending=[False, False, True, False])
print(train_times_table)

['/Users/gmelli/Documents/GitHub_public/notebooks_Jupyter/Text_Data/../../gensim/test/test_data/lee_background.cor', '/Users/gmelli/Documents/GitHub_public/notebooks_Jupyter/Text_Data/text8_50000000']


TypeError: __init__() got an unexpected keyword argument 'compute_loss'

## Conclusion

In this tutorial we learned how to train word2vec models on your custom data and also how to evaluate it. Hope that you too will find this popular tool useful in your Machine Learning tasks!

## Links


Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).